# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Load data generated  from Part I ( event_datafile_new.csv) into Apache Cassandra . 

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code in Part I above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
#  Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

#### Set Keyspace

In [6]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

## Create queries for the following three questions of the data ::

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [7]:
## Query 1: 
# Search the music app history logs for artist and song details that was heard during a particular session and item in session.
# Ex: Give the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

#Query looks something like this.
#SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4

##Create the table with the partition key as the session ID and the clustering key as itemInSession for uniqueness.

session.execute("""CREATE TABLE IF NOT EXISTS musichistory_searchby_itemsession (sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY(sessionId, itemInSession))""")
                    

In [8]:
#Now load the data into the musichistory_searchby_itemsession table from the csv log data.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO musichistory_searchby_itemsession (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

In [9]:
## Run a  SELECT statement to verify the data was entered into the table and queries as per search criteria.
rows = session.execute("""SELECT artist, song_title, song_length  FROM musichistory_searchby_itemsession WHERE sessionId = 338 AND itemInSession = 4""")

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [10]:
## Query 2: 
#Search music app history logs for ame of artist, song (sorted by itemInSession) and user (first and last name) by userid and session ID.
#Eg: Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#Query looks something like this
#SELECT itemInSession, artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182

#Create the table.
#Our primary key will consist of composite partition key userId, sessionId for uniqueness.
#Our clustering key will be itemInSession so that our results are ordered by it.
#The columns of our table will be: userId, sessionId, itemInSession, artist, song and firstName and lastName

session.execute("""
    CREATE TABLE IF NOT EXISTS musichistory_searchby_usersession
    (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, 
    PRIMARY KEY((userId, sessionId), itemInSession))
    """)

file = 'event_datafile_new.csv'

#Load the data into musichistory_searchby_usersession from the csv file.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO musichistory_searchby_usersession (userId, sessionId,  itemInSession, artist, song, firstName, lastName )"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), int(itemInSession),artist, song, firstName, lastName ))

#Query the table based on the userid and session id to validate the results.
rows = session.execute("""SELECT itemInSession, artist, song, firstName, lastName FROM musichistory_searchby_usersession WHERE userId = 10 AND sessionId = 182""")

for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)           

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [11]:
##Query 3: 
#Search music app history logs for user first and last name who listened to a particular song.
#Eg: Give every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'
#Sample QUERY
#SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'

#Create table with song as the partition key and userId for uniqueness.

session.execute("""
    CREATE TABLE IF NOT EXISTS musichistory_searchby_song
    (song text, userId int, firstName text, lastName text, 
    PRIMARY KEY(song, userId))
    """)
                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO musichistory_searchby_song (song, userId, firstName, lastName )"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (song, int(userId) , firstName, lastName ))
        

In [13]:
rows = session.execute("""SELECT firstName, lastName FROM musichistory_searchby_song WHERE song = 'All Hands Against His Own'""")

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [14]:
## Drop the table before closing out the sessions
session.execute("""DROP TABLE musichistory_searchby_song""")



In [15]:
session.execute("""DROP TABLE musichistory_searchby_usersession""")

session.execute("""DROP TABLE musichistory_searchby_itemsession""")

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()